
**Sample ID:** CES_synthetic_account_management_2.2_conversation_174_12

**Query:** Yes, please tell me about the promotions.

**DB Type:** Base Case

**Case Description:**

**Global/Context Variables:**

**Datetime Context Variables:**

**APIs:**
- ces_account_management

**Databases:**

# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

VERSION = "0.1.4_ces_account_management"  # Pass the version of the API
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Cleanup
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Auth
auth.authenticate_user()
drive_service = build('drive', 'v3')

def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(output_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None
try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    for file in results.get('files', []):
        if file['name'].lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file['id']
            print(f"Found: {file['name']} (ID: {apis_file_id})")
            break
except Exception as e:
    print(f"Error listing files: {e}")

if not apis_file_id:
    sys.exit(f"❌ APIs zip V{VERSION} not found.")

print(f"Downloading APIs zip {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH)

print(f"Extracting {ZIP_PATH}...")
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    for member in zip_ref.namelist():
        if any(member.startswith(p) for p in ITEMS_TO_EXTRACT):
            zip_ref.extract(member, CONTENT_DIR)

os.remove(ZIP_PATH)

if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)

for p in [APIS_DIR, DBS_DIR, SCRIPTS_DIR]:
    print(f"{'✅' if os.path.exists(p) else '❌'} {p}")

## Install Dependencies and Clone Repositories

In [ ]:

!pip install uv
!uv pip install -r /content/APIs/requirements.txt
!uv pip install jsonpath_ng


# Import APIs and initiate DBs

In [ ]:
# import APIs and initialize DBs
from ces_account_management import *
import ces_account_management

def download_json_from_gdrive(gdrive_url: str, output_filename: str = "CESInitialDB.json"):
    match = re.search(r"/d/([a-zA-Z0-9_-]+)", gdrive_url)
    if not match:
        raise ValueError("Invalid Google Drive link format.")
    file_id = match.group(1)

    # Download the file
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_filename, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    print(f"Downloaded: {output_filename}")


gdrive_link = "https://drive.google.com/file/d/1r6s2dREi46wMTUTTpQnSvbpqFc-XUCwK/view"
download_json_from_gdrive(gdrive_link)

ces_account_management.load_state('CESInitialDB.json')

get_customer_account_details(**{'accountId': '55512388'})
check_device_upgrade_eligibility(**{'identifierType': 'LINE_NUMBER', 'identifier': '555-4444', 'accountId': '55512388'})

# Initial Assertion

# Action

In [ ]:
#proto_ignore
from ces_account_management import *

In [ ]:
query_available_plans_and_features(**{'query': 'current device upgrade promotions'})

# Golden Answer

You already have our best plan, but we do have a couple of add-ons that would be a great value with your new phone. We have Device Protection Plus, which covers accidental damage and extended warranty for one device, and it's only \$11.99 a month. We also have an International Calling Pack that gives you 100 minutes to select international destinations for \$10 a month." Is there anything else I can help you with today?

# Final Assertion